In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [14]:

pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Rc_884/COMPINT'
nx=256
ny=384
ncomp=4


'''
pref = '/home/saberbud/Programs/Paper_RB_14bit_COMPINTs/CI16164/COMPINT'
nx=128
ny=192
ncomp=4
'''

'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'
nx=64
ny=96
ncomp=4
'''

"\npref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_32324/COMPINT'\nnx=64\nny=96\nncomp=4\n"

In [15]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 256, 384, 4) , mean = 154.70262400309244 , std = 12.840658757447757 , Max = 200.0 , Min = 118.0 , type=float64
t_C_0 shape: (1, 256, 384) , mean = 152.05585734049478 , std = 10.799949008841688 , Max = 193.0 , Min = 118.0 , type=float64
t_C_1 shape: (1, 256, 384) , mean = 148.60657755533853 , std = 9.025510103642233 , Max = 174.0 , Min = 123.0 , type=float64
t_C_2 shape: (1, 256, 384) , mean = 156.33193969726562 , std = 12.254867510801619 , Max = 196.0 , Min = 122.0 , type=float64
t_C_3 shape: (1, 256, 384) , mean = 161.81612141927084 , std = 14.622786642373688 , Max = 200.0 , Min = 126.0 , type=float64


In [16]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (256, 384) , mean = 33.816121419270836 , std = 14.622786642373688 , Max = 72 , Min = -2 , type=int8


In [17]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen98304
size198316
size265740
ratio1= 1.00012207031
ratio2= 0.668741861979


In [18]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 98304
mse = 0.0


In [19]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 256, 384, 4) , mean = 155.35005950927734 , std = 12.8632464201734 , Max = 217.0 , Min = 116.0 , type=float64
 channel0 shape: (256, 384) , mean = 24.6290283203125 , std = 11.073007955186805 , Max = 65 , Min = -12 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 62759 Bytes
ratio1= 1.00012207031
ratio2= 0.638417561849
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 21.044382731119793 , std = 9.03525524727801 , Max = 55 , Min = -8 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 58895 Bytes
ratio1= 1.00012207031
ratio2= 0.599110921224
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 29.056193033854168 , std = 12.204057591839094 , Max = 85 , Min = -4 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 63589 Bytes
ratio1= 1.00012207031
ratio2= 0.646860758464
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 34.670633951822914 , std = 14.317779068903935 , Max = 89 , Min = 0 , type=int8
   olen: 

dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 26.36614990234375 , std = 10.402102839993717 , Max = 58 , Min = -5 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 65723 Bytes
ratio1= 1.00012207031
ratio2= 0.668568929036
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 36.21613566080729 , std = 13.954656200108865 , Max = 78 , Min = -1 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 70703 Bytes
ratio1= 1.00012207031
ratio2= 0.719228108724
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 43.204488118489586 , std = 16.209670940683218 , Max = 84 , Min = 0 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 73088 Bytes
ratio1= 1.00012207031
ratio2= 0.743489583333
dlen= 98304
mse = 0.0



9
  t_C shape: (1, 256, 384, 4) , mean = 166.16130828857422 , std = 11.000239673264984 , Max = 204.0 , Min = 119.0 , type=float64
 channel0 shape: (256, 384) , mean = 34.426025390625 , std = 7.707562199674766 , Max = 63 , Min = 

   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 62310 Bytes
ratio1= 1.00012207031
ratio2= 0.633850097656
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 32.48876953125 , std = 25.864014406639043 , Max = 82 , Min = -3 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 64882 Bytes
ratio1= 1.00012207031
ratio2= 0.660013834635
dlen= 98304
mse = 0.0



16
  t_C shape: (1, 256, 384, 4) , mean = 157.52423350016275 , std = 12.491533293786942 , Max = 199.0 , Min = 118.0 , type=float64
 channel0 shape: (256, 384) , mean = 26.592681884765625 , std = 10.276033207088433 , Max = 53 , Min = -10 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 54961 Bytes
ratio1= 1.00012207031
ratio2= 0.559092203776
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 22.782877604166668 , std = 8.50970129137049 , Max = 44 , Min = -2 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 50839 Bytes
ratio1= 1.00012207031
ratio2= 0.517161051432
dlen= 98304


dlen= 98304
mse = 0.0



23
  t_C shape: (1, 256, 384, 4) , mean = 150.329714457194 , std = 14.880876153078498 , Max = 210.0 , Min = 117.0 , type=float64
 channel0 shape: (256, 384) , mean = 20.10809326171875 , std = 12.769652296972424 , Max = 64 , Min = -3 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 63671 Bytes
ratio1= 1.00012207031
ratio2= 0.647694905599
dlen= 98304
mse = 0.0

 channel1 shape: (256, 384) , mean = 17.212605794270832 , std = 10.934320845116572 , Max = 54 , Min = -11 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 60977 Bytes
ratio1= 1.00012207031
ratio2= 0.620290120443
dlen= 98304
mse = 0.0

 channel2 shape: (256, 384) , mean = 23.685618082682293 , std = 14.886682371860516 , Max = 69 , Min = -3 , type=int8
   olen: 98304 Bytes
   size1: 98316 Bytes
   size2: 66190 Bytes
ratio1= 1.00012207031
ratio2= 0.673319498698
dlen= 98304
mse = 0.0

 channel3 shape: (256, 384) , mean = 28.312540690104168 , std = 17.68536428755981 , Max = 82 , M

In [20]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.0001220703125
   ratio_2= 0.6389389038085938
 
2
   ratio_1= 1.0001220703125
   ratio_2= 0.4737396240234375
 
3
   ratio_1= 1.0001220703125
   ratio_2= 0.5730845133463541
 
4
   ratio_1= 1.0001220703125
   ratio_2= 0.6439412434895834
 
5
   ratio_1= 1.0001220703125
   ratio_2= 0.6842142740885416
 
6
   ratio_1= 1.0001220703125
   ratio_2= 0.5979741414388021
 
7
   ratio_1= 1.0001220703125
   ratio_2= 0.6298243204752604
 
8
   ratio_1= 1.0001220703125
   ratio_2= 0.7066726684570312
 
9
   ratio_1= 1.0001220703125
   ratio_2= 0.5956268310546875
 
10
   ratio_1= 1.0001220703125
   ratio_2= 0.6125996907552084
 
11
   ratio_1= 1.0001220703125
   ratio_2= 0.5861943562825521
 
12
   ratio_1= 1.0001220703125
   ratio_2= 0.5346298217773438
 
13
   ratio_1= 1.0001220703125
   ratio_2= 0.6520131429036459
 
14
   ratio_1= 1.0001220703125
   ratio_2= 0.6680577596028646
 
15
   ratio_1= 1.0001220703125
   ratio_2= 0.6217931111653646
 
16
   ratio_1= 1.0001220703125
   ratio_2= 0.5617